In [1]:
import os
import re

import numpy as np
import pandas as pd
import tables

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, r2_score, mean_absolute_error, mean_squared_error

from gensim.models import FastText

from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from tensorflow.keras import regularizers

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import plot_model

/opt/ohpc/pub/apps/python3/3.8.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-30 20:17:16.962905: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from tensorflow.keras import regularizers
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

!pip install LibRecommender
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF
from libreco.evaluation import evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
books_df = pd.read_csv('/home/016709732/dataset/goodreads_interactions.csv')

In [3]:
books_df = books_df.drop(books_df[books_df['is_read'] == 0].index)
books_df = books_df.drop(books_df[books_df['is_reviewed'] == 0].index)
books = books_df.groupby('rating', group_keys=False).apply(lambda x: x.sample(int((len(x)/len(books_df))*500000)))
books.shape

(499998, 5)

In [4]:
books.head(20)

,user_id,book_id,is_read,rating,is_reviewed
48097271,95527,1571711,1,0,1
10699894,20905,91664,1,0,1
9721705,19042,416189,1,0,1
22640665,44472,1320,1,0,1
225699884,798440,19431,1,0,1
225903032,804054,76073,1,0,1
45964627,91432,181059,1,0,1
181877438,382787,444801,1,0,1
175615775,367881,583544,1,0,1
175346382,367192,354395,1,0,1


In [5]:
books["user"] = books["user_id"]
books["label"] = books["rating"]

books["item"] = books["book_id"]
books = books[["user", "item", "label"]]

In [8]:
train_data, eval_data, test_data = random_split(books, multi_ratios=[0.8, 0.1, 0.1])

In [10]:
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [11]:
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=5,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

2023-10-30 20:21:58.612463: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-10-30 20:21:58.612661: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-10-30 20:21:58.785645: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-30 20:21:58.785700: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c4): /proc/driver/nvidia/version does not exist


In [12]:
ncf.fit(
    train_data,
    neg_sampling=False,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)

Training start time: 2023-10-30 20:22:08


/home/016709732/.local/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/normalization.py:307: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  warnings.warn(
/home/016709732/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-10-30 20:22:09.247150: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2023-10-30 20:22:09.263332: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1995290000 Hz


Epoch 1 elapsed: 5.177s
	 train_loss: 3.444


eval_pointwise: 100%|█████████████████████████████| 3/3 [00:00<00:00, 56.25it/s]


	 eval rmse: 1.1549


...

	 eval rmse: 1.1793


train: 100%|██████████████████████████████████| 391/391 [00:04<00:00, 83.79it/s]


Epoch 5 elapsed: 4.669s
	 train_loss: 0.3402


eval_pointwise: 100%|████████████████████████████| 3/3 [00:00<00:00, 166.51it/s]


	 eval rmse: 1.1698


In [13]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)

eval_pointwise: 100%|████████████████████████████| 3/3 [00:00<00:00, 133.04it/s]


{'loss': 1.175301}

In [14]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["mae"],
)

eval_pointwise: 100%|████████████████████████████| 3/3 [00:00<00:00, 122.00it/s]


{'mae': 0.90626866}

In [15]:
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["rmse"],
)

eval_pointwise: 100%|████████████████████████████| 3/3 [00:00<00:00, 146.70it/s]


{'rmse': 1.175301}